In [ ]:
import os
import subprocess
import argparse
import string
import json
import datetime
from BSpline_Transform import deformable_transform
from Affine_Transform import affine_transform
from util import find_number, Euc_dist

# Image Registration

In [ ]:
# This is folder where the nifti files of images that are undergoing registration are saved

Folder_Of_Image = 'Ada_NiftiChallenge'

### Setting up B-Spline and Affine Parameters

The following cell enables setting up parameters for B_Spline and Affine before creating the parameter files that will be used for the registration.

In case of B_Spline following parameters can be optimized
- Number of Resolutions
- Final Grid Spacing
- Spacing Schedule
- Pyramid Schedule
- Number of Iterations
- Number of spatial samples used for metric evaluation
- Number of Histogram Bins
- Metric that will be used

For the Affine transformation the parameters that are optimized include
- Number of Resolutions
- Number of Iterations
- Number of spatial samples used for metric evaluation
- Metric 
- Number of Histogram Bins


In [ ]:
# B_Spline Parameters
RESOLUTION = '6'
F_GRID_SPACING = '10'
ITER_NUMBER = '1000'
SPATIAL_SAMPLES = '2000'
SPACING_SCHEDULE =  '16.0 8.0 8.0 4.0 2.0 1.0'
PYRAMID_SCHEDULE = '16 16 16 8 8 8 4 4 4 2 2 2 1 1 1 1 1 1'
BIN_NUMBER = '32'
FILL_VALUES = '0'
METRIC = 'AdvancedNormalizedCorrelation'

# Affine Parameters
A_RESOLUTION = '6'
A_ITER_NUMBER = '1000'
A_SPATIAL_SAMPLES = '2000'
A_METRIC = 'AdvancedNormalizedCorrelation'
A_BIN_NUMBER = '32'

Set up the foldere where the parameter files will be saved

In [ ]:
OUTDIR_PARAM = 'Parameters_Folder/'
os.makedirs(OUTDIR_PARAM, exist_ok=True)

## Creation of registration parameter files

In the following cell the respective commands for b-spline [deformable_transform] and affine [affine_transform] functions are called for creation of '.txt' files that will be used for image registration. 

In [ ]:
deform_transfrom, b_name = deformable_transform(name = '0', resolution = RESOLUTION, f_grid_spacing = F_GRID_SPACING, iter_number = ITER_NUMBER, 
                                       spatial_samples = SPATIAL_SAMPLES, spacing_schedule = SPACING_SCHEDULE, 
                                       pyramid_schedule = PYRAMID_SCHEDULE, metric = METRIC, bin= BIN_NUMBER, fill_value=FILL_VALUES)

aff_transform, a_name = affine_transform(name = '0', resolution = A_RESOLUTION, iter_number=A_ITER_NUMBER, 
                                        spatial_samples = A_SPATIAL_SAMPLES, metric = A_METRIC, bin = A_BIN_NUMBER)

Save the '.txt' files 

In [ ]:
out = '%d_%s.txt' % (0, b_name)
out = os.path.join(OUTDIR_PARAM, out)
with open(out, 'w') as file:
    file.write(deform_transfrom.rstrip())

out = '%d_%s.txt' % (0, a_name)
out = os.path.join(OUTDIR_PARAM, out)
with open(out, 'w') as file:
    file.write(aff_transform.rstrip())

Set up the name and location of the parameter files that will be called in the following cell

In [ ]:
name1 = f"{OUTDIR_PARAM}/0_{a_name}.txt"
name2 = f"{OUTDIR_PARAM}/0_{b_name}.txt"

only_BSpline = 'True'

By using only_BSpline set to True or False user can decide if only B-Spline or both B-Spline and affine are used for image registration. In case that only_BSpline is set to 'True' the elastix command will only make a call to the parameter file of the B-spline registration. If it is set to 'False' the elastix command will firstly call the affine parameter file and then the B-spline parameter file. In case the it is not defined the for loop will not proceedd to run.

string1 is used for calling the elastix command
string2 is used for calling the transformix function and transforming the landmarks with respect to obtained Transformation Parameters


In [ ]:
for k in [0, 5, 6]:
    OUTDIR = f'logs/{k}'
    # make the output directory for the files that generated during image registration
    os.makedirs(OUTDIR, exist_ok=True)
    if only_BSpline == 'False':
        string1  = f"-f ./{Folder_Of_Image}/copd{k}_iBHCT.nii.gz -m ./{Folder_Of_Image}/copd{k}_eBHCT.nii.gz -out ./{OUTDIR}/ -p {name1} -p {name2}"
        string2 = f"-def ChallengeData/copd{k}/copd{k}_300_iBH_xyz_r1.txt -out ./{OUTDIR}/ -tp ./{OUTDIR}/TransformParameters.1.txt"
    
    elif only_BSpline == 'True'
        string1  = f"-f ./{Folder_Of_Image}/copd{k}_iBHCT.nii.gz -m ./{Folder_Of_Image}/copd{k}_eBHCT.nii.gz -out ./{OUTDIR}/ -p {name2}"
        string2 = f"-def ChallengeData/copd{k}/copd{k}_300_iBH_xyz_r1.txt -out ./{OUTDIR}/ -tp ./{OUTDIR}/TransformParameters.0.txt"
    else:
        print('The exact transformation procedure is not defined')
        break
    
    !elastix {string1}
    !transformix {string2}

# TRE Calculation 

As every volume has a specific spacing it has to be set up in advance before using it

In [ ]:
n = 1
spacing = [(0.625,0.625,2.5),(0.645,0.645,2.5),(0.652,0.652,2.5),(0.590,0.590,2.5)];

In [ ]:
i = 0
for i in range(4):
    n = i + 1
    print(n)
    OUTDIR = f'logs/{n}'
    data = pd.read_csv(f'./{OUTDIR}/outputpoints.txt', sep=";", header=None)
    outputData = data[3]
    check = []
    
    # Extract the registered landmarks, only the number value and 
    # transform them into a numpy array of 'float64' dtype. 
    for k in range(300):
        values = find_number(outputData[k])
        check.append(values.split())
    check = np.array(check)
    check = np.squeeze(check)
    check = check.astype('float64')
    
    landmark1= np.loadtxt(f'exhale/copd{i+1}_300_eBH_xyz_r1.txt')
    landmark2 = check
    dist, mean,std = Euc_dist(landmark1,landmark2,spacing[i])
    print("results of "+str(i+1))
    print(mean, std)